In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
#미리 저장해둔 모델 로딩
import tensorflow

model = tensorflow.keras.models.load_model("/gdrive/MyDrive/Colab Notebooks/capstone/best_model_4.h5") 

In [ ]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [ ]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          '끝내려면 여기를 클릭하세요.</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, overay_img):
  data = eval_js('stream_frame("{}", "{}")'.format(label, overay_img))
  return data

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class Classifier:

    def classify(self, image):
        
        predicted = model.predict(np.expand_dims(image, axis=0))
        # return np.argmax(predicted, axis=1)[0]
        return (np.argmax(predicted[0]), np.max(predicted[0]), predicted[0])
classifier = Classifier()

In [ ]:
# start streaming video from webcam
video_stream()

# label for video
result = 'Capturing...'

# 1부터 10까지 클래스별로 숫자 지정
overray_bytes = ''
count = 0 
class_name = { 0:"1", 1:"10", 2:"10", 3:"2", 4:"3", 5:"4", 6:"5", 7:"6", 8:"7", 9:"8", 10:"9"}

while True:
    js_reply = video_frame(result, overray_bytes)
    if not js_reply:
        break
    # convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])
    img = cv2.resize(img, (224, 224))




    # category = classifier.classify(img)
    
    category, confidence, confidence_vector = classifier.classify(img)
    print("number : {}, confidence : {}".format(class_name[category], confidence))


    # create transparent overlay for bounding box
    overray_img = np.zeros([480,640,4], dtype=np.uint8)
    overray_img = cv2.putText(overray_img, class_name[category], (240,320), cv2.FONT_HERSHEY_SIMPLEX, 10, 255)

    # convert overlay of bbox into bytes
    overray_bytes = bbox_to_bytes(overray_img)
    result = class_name[category]
    


<IPython.core.display.Javascript object>

number : 10, confidence : 0.9999915361404419
number : 1, confidence : 0.999881386756897
number : 7, confidence : 0.5148602724075317
number : 1, confidence : 0.9999980926513672
number : 10, confidence : 0.9622534513473511
number : 10, confidence : 1.0
number : 10, confidence : 0.9999825954437256
number : 10, confidence : 0.9948222637176514
number : 10, confidence : 0.9897679686546326
number : 5, confidence : 0.5405893921852112
number : 7, confidence : 0.45628461241722107
number : 7, confidence : 0.5592954754829407
number : 10, confidence : 0.9863843321800232
number : 7, confidence : 0.5836915373802185
number : 7, confidence : 0.38987645506858826
number : 7, confidence : 0.9980937838554382
number : 10, confidence : 0.9993971586227417
number : 10, confidence : 0.999718964099884
number : 5, confidence : 0.32206597924232483
number : 5, confidence : 0.5062727928161621
number : 5, confidence : 0.4591890573501587
number : 10, confidence : 0.5271917581558228
number : 5, confidence : 0.465169608